In [6]:
import os
# file paths
directory = os.getcwd()
DATA_DIR = os.path.abspath("../data")
TRAIN_PATH = os.path.join(DATA_DIR, 'stage1_train')
TEST_PATH = os.path.join(DATA_DIR, 'stage1_test')
INPUT_PATH = os.path.join(directory, 'input')

In [20]:
import skimage.io
import numpy as np
def read_image_labels(image_id):
    image_file = os.path.join(TRAIN_PATH, "{}", "images", "{}.png").format(image_id, image_id)
    mask_file = os.path.join(TRAIN_PATH,  "{}", "masks", "*.png").format(image_id)
    image = skimage.io.imread(image_file)
    masks = skimage.io.imread_collection(mask_file).concatenate()
    height, width, _ = image.shape
    num_masks = masks.shape[0]
    labels = np.zeros((height, width), np.uint16)
    for index in range(0, num_masks):
        labels[masks[index] > 0] = index + 1
    return image, labels

In [25]:
from skimage import transform
def data_aug(image, label, angle=30, resize_rate=0.9):
    flip = random.randint(0, 1)
    size = image.shape[0]
    rsize = random.randint(np.floor(resize_rate*size),size)
    w_s = random.randint(0,size - rsize)
    h_s = random.randint(0,size - rsize)
    sh = random.random()/2-0.25
    rotate_angle = random.random()/180*np.pi*angle
    # Create Affine transform
    affine_tf = transform.AffineTransform(shear=sh, rotation=rotate_angle)
    # Apply transform to image data
    image = transform.warp(image, inverse_map=affine_tf, mode='edge')
    label = transform.warp(label, inverse_map=affine_tf, mode='edge')
    # Randomly cropping image frame
    image = image[w_s:w_s+size, h_s:h_s+size, :]
    label = label[w_s:w_s+size, h_s:h_s+size]
    # Randomly flip frame
    if flip:
        image = image[:, ::-1, :]
        label = label[:, ::-1]
    return image, label